# LeNet5

## Models

![LeNet5](lenet5.png)

- Does not scale well (uses hyperbolic tangents after every linear layer)
- Killed off the gradients, (activation will saturate since sigmoid function is used)

In [1]:
import tensorflow as tf
from run_model import *
from utils.cifar10_utils import *
from models_utils import *

In [2]:
DATA_DIR = "/home/karen/workspace/data/cifar10_data"
BATCH_SIZE = 128
NUM_EPOCHS = 1000

In [3]:
def lenet_model(features, training, use_batchnorm, data_format, name="lenet5"):
    # Inputs should be images with dimension (H, W)
    inputs = features
    inputs = tf.layers.conv2d(inputs,6,5,padding='valid',data_format='channels_last')
    # C1
    inputs = tf.layers.max_pooling2d(inputs,2,2)
    # S2
    
    inputs = tf.layers.conv2d(inputs,16,5,padding='valid',data_format='channels_last')
    # C3
    
    inputs = tf.layers.max_pooling2d(inputs,2,2)
    # S4
    
    inputs = tf.layers.conv2d(inputs,120,5,padding='valid',data_format='channels_last')
    # C3
    
#     inputs = tf.squeeze(inputs)
    inputs = tf.layers.flatten(inputs)
    
    inputs = tf.layers.dense(inputs, 84)
    
    return tf.layers.dense(inputs, 10)

In [4]:
input_train_fn = lambda:input_fn(is_training=True, data_dir=DATA_DIR, 
                          batch_size=per_device_batch_size(BATCH_SIZE, 1),
                          num_epochs=NUM_EPOCHS,
                          num_gpus=1)

In [5]:
model_fn = lambda features, labels, mode :resnet_model_fn(features, labels, mode, 
                                                          lenet_model, "classic_lenet5_bn", True, 
                                                          False, "channels_first")

In [6]:
# distribution = tf.contrib.distribute.OneDeviceStrategy('device:GPU:0')
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
run_config = tf.estimator.RunConfig(session_config=config)
classifier = tf.estimator.Estimator(model_fn=model_fn, 
                                    model_dir="./tmp/classic_lenet5_bn",
                                    config=run_config,
                                    params=None)

INFO:tensorflow:Using config: {'_model_dir': './tmp/classic_lenet5_bn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  allow_growth: true
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7caf84e908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
classifier.train(input_fn=input_train_fn, hooks=[],
                 max_steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/classic_lenet5_bn/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2 into ./tmp/classic_lenet5_bn/model.ckpt.
INFO:tensorflow:loss = 2.324249, step = 1
INFO:tensorflow:Saving checkpoints for 10 into ./tmp/classic_lenet5_bn/model.ckpt.
INFO:tensorflow:Loss for final step: 3.0199363.
